参考：
- [Running Deep Learning models in OpenCV](http://cv-tricks.com/how-to/running-deep-learning-models-in-opencv/?utm_campaign=dep-learning-models-in-op&utm_source=sendpulse&utm_medium=email)
- [Darknet yolo](https://pjreddie.com/darknet/yolo/)

 目前，OpenCV 支持 Caffe，Torch，Tensorflow 和 Darknet。如果您在生产中部署了基于 OpenCV 的模型，例如 Hog + SVM 分类器或基于 Haar 级联的检测器等，并且您希望将其替换为更准确的适当的基于深度学习的模型，则此功能尤其有用。因此，您可以在自己喜欢的深度学习库中训练模型，并在生产基础架构中部署该模型，而不会中断当前的工作流程。在本系列教程中，我们将学习如何将在其他框架中训练的模型部署到 openCV。特别是在这一个，我们将学习：

- 如何在 OpenCV 中部署基于 Darknet 的对象检测模型。我们将部署 Yolov2 并在一些图像和视频上运行它。
- 在第 $2$ 部分中，我们将部署Sqeezenet模型到openCV以生成预测。
- 在第 $3$ 部分中，我们将在 OpenCV 中部署 Tensorflow 模型。

# 在 OpenCV 中部署 YOLO-V2 模型

```py
import cv2
model = 'yolov2.weights'
config = 'yolov2.cfg'
net = cv2.dnn.readNetFromDarknet(config, model)
```
This little command will give us the network architecture as specified in config loaded with the trained weights of yolov2.

## blobFromImage method
下一步是批量加载图像并通过网络运行它们。为此，我们使用 `cv2.dnn.blobFromImage` 方法。此方法从输入图像创建 $4$ 维 `blob`。我们来看看这个方法的签名：`blob = cv.dnn.blobFromImage(image, scalefactor, size, mean, swapRB, crop)`

- `image`：是我们想要发送到神经网络进行推理的输入图像。
- `scalefactor`：如果我们想通过将它们乘以常数来缩放我们的图像。很多时候我们将所有 `uint8` 图像除以 $255$，这样所有像素都在 $0$ 到 $1$ 之间。默认值为 $1.0$，没有缩放。
- `size`：输出图像的空间大小。它将等于后续神经网络作为 `blobFromImage` 输出所需的输入大小。
- `swapRB`：布尔值，表示我们是否要交换 $3$ 通道图像中的第一个和最后一个通道。OpenCV 假定图像默认为 BGR 格式，但如果我们想将此顺序交换为RGB，我们可以将此标志设置为 `True`，这也是默认值。
- `mean`：为了处理强度变化和归一化，有时我们计算训练数据集上的平均像素值，并在训练期间从每个图像中减去它。如果我们在训练期间进行平均减法，那么我们必须在推理期间应用它。这意味着将是对应于 R，G，B 通道的元组。例如，imagenet 数据集上的平均值是 $R = 103.93，G = 116.77，B = 123.68$。如果我们使用 `swapRB = False`，那么这个顺序将是 `(B, G, R)`。
- `crop`：布尔标志，指示我们是否要对裁剪图像进行居中。如果将其设置为 `True`，则输入图像将从中心裁剪，使得较小的尺寸等于相应的尺寸尺寸，而其他尺寸等于或大于此尺寸。但是，如果我们将其设置为 `False`，它将保留纵横比并仅调整尺寸大小。

YOLO-V2 模型需要大小为 (416,416) 的输入图像，并通过将它们除以 $255$ 来缩放图像。因此，我们使用它的方式如下：
`blob = cv.dnn.blobFromImage(frame, 1.0/255.0, (416, 416), True, crop=False)`

最后，我们得到输出预测并在输入图像上绘制它们。

```py
net.setInput(blob)
# Run the preprocessed input blog through the network
predictions = net.forward()
probability_index=5

for i in range(predictions.shape[0]):
    prob_arr=predictions[i][probability_index:]
    confidence=np.amax(prob_arr)
    if confidence > min_confidence:
        x_center=predictions[i][0]*width
        y_center=predictions[i][1]*height
        width_box=predictions[i][2]*width
        height_box=predictions[i][3]*height
        
        x1=int(x_center-width_box * 0.5)
        y1=int(y_center-height_box * 0.5)
        x2=int(x_center+width_box * 0.5)
        y2=int(y_center+height_box * 0.5)
     
        cv.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)
cv.imshow(winName, frame)
```

这篇文章的代码可以在 github 上的[代码库](https://github.com/sankit1/cv-tricks.com/tree/master/OpenCV/Running_YOLO)中找到。